In [1]:
% matplotlib inline
from __future__ import division

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/Users/sclayton/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [ ]:
def checklon(data):
    cxx = (data['lon']).values # longitude

    # convert all longitude to longitude east
    cxx[cxx<0]= 360+cxx[cxx<0];
    data['lon_e'] = cxx
    cxbew = cxx
    cxbew[cxbew>180] = cxbew[cxbew>180]-360   
    data['lon'] = cxbew
    return data

In [2]:
# import the data
# pro ecotypes
filePath = r'/Users/sclayton/Google Drive/work/physics_traits/data/Monster dataset AMT 13_proportal.xlsx'
pro_table = pd.read_excel(filePath)

# syn ecotypes
filePath = r'/Users/sclayton/Google Drive/work/physics_traits/data/Syne data for Sophie.xlsx'
syn_table = pd.read_excel(filePath)

In [5]:
print pro_table.columns
print syn_table.columns

Index([u'CTD', u'lat', u'lon', u'JD (GMT)', u'CTD deploy (GMT)', u'Niskin #',
       u'depth (m)', u'Pro FCM', u'T QPCR', u'MED4', u'MIT9312', u'MIT9211',
       u'NATL2A', u'SS120', u'MIT9313', u'Syn FCM', u'TEMP', u'light'],
      dtype='object')
Index([u'sample/cruise', u'station', u'lat (deg N)', u'lon', u'depth', u'temp',
       u'syne1 (cells/ml)', u'syne2', u'syne3', u'syne4', u'syne8', u'syne10',
       u'syne15', u'syne16', u'CRD1', u'CRD2'],
      dtype='object')
